# Setup

In [1]:
#@title Dependencies and Imports
%load_ext autoreload
%autoreload 2
%pip install -q lightning click transformers goatools toml wget fastobo torch_xla wandb

from google.colab import userdata, drive
import os
from pathlib import Path


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 11.7 MB/s eta 0:00:00


In [2]:
#@title Environment Variables
github_pat = userdata.get("GITHUB_PAT")
wandb_key = userdata.get("WANDB_API_KEY")
os.environ["WANDB_API_KEY"] = wandb_key
drive.mount("/content/drive")


Mounted at /content/drive


In [3]:
#@title Clone and cd
if os.getcwd() != "/content/contempro/work":
  if not Path("/content/contempro").exists():
    !git clone https://{github_pat}@github.com/boun-tabi-lifelu/contempro.git
  %cd /content/contempro/work

Cloning into 'contempro'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 269 (delta 128), reused 220 (delta 81), pack-reused 0 (from 0)
Receiving objects: 100% (269/269), 6.37 MiB | 46.60 MiB/s, done.
Resolving deltas: 100% (128/128), done.
/content/contempro/work


# Data Fetching

In [4]:
#@title Setup data
!mkdir -p datasets
!cp /content/drive/MyDrive/research/contempro/work-pfresgo-data.zip ./datasets
!cd datasets && unzip -o work-pfresgo-data.zip

Archive:  work-pfresgo-data.zip
   creating: pfresgo/
  inflating: __MACOSX/._pfresgo      
  inflating: pfresgo/.DS_Store       
  inflating: __MACOSX/pfresgo/._.DS_Store  
  inflating: pfresgo/annot.tsv       
  inflating: __MACOSX/pfresgo/._annot.tsv  
  inflating: pfresgo/nrPDB-GO_2019.06.18_test.csv  
  inflating: __MACOSX/pfresgo/._nrPDB-GO_2019.06.18_test.csv  
  inflating: pfresgo/train.txt       
  inflating: __MACOSX/pfresgo/._train.txt  
  inflating: pfresgo/go.obo          
  inflating: __MACOSX/pfresgo/._go.obo  
  inflating: pfresgo/ontology.embeddings.npy  
  inflating: __MACOSX/pfresgo/._ontology.embeddings.npy  
  inflating: pfresgo/valid.txt       
  inflating: __MACOSX/pfresgo/._valid.txt  
  inflating: pfresgo/nrPDB-GO_2019.06.18_sequences.fasta  
  inflating: __MACOSX/pfresgo/._nrPDB-GO_2019.06.18_sequences.fasta  
  inflating: pfresgo/test.txt        
  inflating: __MACOSX/pfresgo/._test.txt  


In [5]:
!cp /content/drive/MyDrive/research/per_residue_embeddings.h5 ./datasets/pfresgo

# Test DataModule

In [ ]:
%load_ext autoreload
%autoreload 2
from data.datamodule import PFresGODataModule

dm = PFresGODataModule(
    data_dir="./datasets/pfresgo",
    ontology="molecular_function",
    num_workers=10
)
dm.setup()
dl = dm.train_dataloader()
next(iter(dl))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/mmtf/p/research/contempro/work/.pixi/envs/default/lib/python3.13/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/Users/mmtf/p/research/contempro/work/.pixi/envs/default/lib/python3.13/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ~~~~~~~~~~~~~~~^^^^^^
  File "/Users/mmtf/p/research/contempro/work/data/datamodule.py", line 236, in collate_fn
    seq_emb_dim = batch[0].residue_embeddings.shape[1]
                  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: tuple index out of range


# Training

In [6]:
%run -i train.py

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


ModuleNotFoundError: No module named 'wandb'

In [11]:
from data.datamodule import PFresGODataset
import h5py
import numpy as np
from typing import List
from pathlib import Path
def _load_protein_ids(filename: Path) -> List[str]:
  """Load protein IDs from a text file"""
  with open(filename) as f:
    return [line.strip() for line in f]


def _load_residue_embeddings(prot_ids, residue_emb_file: Path) -> dict[str, np.ndarray]:
  with h5py.File(residue_emb_file, 'r') as f:
    return {
        # TODO: remove the last dimension
        protein: f[f"{protein} nrPDB"][:]
        for protein in prot_ids
    }

ids = []
for slic in ["train", "valid", "test"]:
  ids += _load_protein_ids(f"datasets/pfresgo/{slic}.txt")

In [14]:
embs = _load_residue_embeddings(ids, "datasets/pfresgo/per_residue_embeddings.h5")

In [2]:
import numpy as np

In [12]:
%cd /content/contempro/work

/content/contempro/work


In [ ]:
np.savez_compressed("embs_2.npz", **embs)